In [ ]:
!pip install transformers torch
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 14.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import numpy as np


with open('data.txt', 'r') as f:
    data = f.read()


samples = data.split('--')


labels = []
texts = []


for sample in samples:
    if sample:
        split_sample = sample.split('***')
        labels.append(split_sample[0].strip())
        clean_text = re.sub(r'[^\w\s]', '', split_sample[1].strip())
        clean_text = re.sub(r'\s+', ' ', clean_text)
        clean_text = re.sub(r'\n', '', clean_text)

        texts.append(clean_text)


df = pd.DataFrame({'label': labels, 'text': texts})

df['label'] = df['label'].str.replace(' ', '')

df = df.sample(frac=1, random_state=42)

from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(df['label'])

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

encodings = tokenizer(df['text'].tolist(), truncation=True, padding=True, max_length=128)

labels = torch.tensor(labels)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

dataset = Dataset(encodings, labels)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(labels)))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Step,Training Loss
500,3.042200


TrainOutput(global_step=700, training_loss=2.2724183654785155, metrics={'train_runtime': 230.0716, 'train_samples_per_second': 42.595, 'train_steps_per_second': 3.043, 'total_flos': 645177714585600.0, 'train_loss': 2.2724183654785155, 'epoch': 100.0})

In [ ]:
new_texts =["""Sulphur is such a full remedy that it is somewhat difficult to tell where to begin.

It seems to contain a likeness of all the sicknesses of man, and a beginner on reading over the proving of Sulphur might naturally think that he would need no other remedy, as the image of all sickness seems to be contained in it.

Yet you will find it will not cure all the sicknesses of man, and it is not well to use it indiscriminately any more than you would any other remedy. It seems that the less a physician knows of the Materia Medica the oftener he gives Sulphur, and yet it is very frequently given, even by good prescribers; so that the line between physicians’ ignorance and knowledge cannot be drawn from the frequency with which Sulphur is prescribed by them.

Aspect: The Sulphur patient is a lean, lank, hungry, dyspeptic fellow with stoop shoulders, yet many times it must be given to fat, rotund, well fed people.

The angular, lean, stoop-shouldered patient, however, is the typical one, and especially when he has become so from long periods of indigestion, bad assimilation and feeble nutrition. The Sulphur state is sometimes brought about by being long housed up and adapting the diet to the stomach.

Persons who lead sedentary lives, confined to their rooms in study, in meditation, in philosophical inquiry, and who take no exercise, soon find out that they must eat only the simplest foods, foods not sufficient to nourish the body, and they end up by going into a philosophical mania.

There is another class of patients in whom we see a Sulphur appearance in the face; dirty, shriveled, red-faced people. The skin seems to be easily affected by the atmosphere. He becomes red in the face from riding in the air, both in very cold and in damp weather.

He has a delicate, thin skin, blushing on the slightest occasion, always red and dirty looking, no matter how much he washes it. If it be a child, the mother may wash the face often, but it always looks as if it had been perfunctorily washed.

Hering called the Sulphur patient "the ragged philosopher."

The Sulphur scholar, the inventor, works day and night in threadbare clothes and battered hat; he has long, uncut hair and a dirty face; his study is uncleanly, it is untidy; books and leaves of books are piled up indiscriminately; there is no order. It seems that Sulfur produces this state of disorder, a state of untidiness, a state of uncleanness, a state of "care how things go," and a state of selfishness.

He becomes a false philosopher and the more he goes on in this state the more he is disappointed because the world does not consider him the greatest man on earth. Old inventors work and work, and fail.

The complaints that arise in this kind of case, even the acute complaints, will run to Sulphur. You take such a patient and you will notice that he has on a shirt that he has worn many weeks; if he has not a wife to attend to him, he would wear his shirt until it fell off from him.

Cleanliness is not a great idea with the Sulphur patient; he thinks it is not necessary. He is dirty; he does not see the necessity of putting on a clean collar and cuffs and a clean shirt; it does not worry him. Sulphur is seldom indicated in cleanly people, but it is commonly indicated in those who are not disturbed by uncleanliness.

Odor: When attending the public clinic I have many times noticed that after Sulphur an individual begins to take notice of himself and puts on a clean shirt, whereas his earlier appearances were in the one same old shirt. And it is astonishing how the Sulphur patients, especially the little ones, can get their clothing dirty so fast.

Children have the most astonishing tendency to be filthy. Mothers tell you of the filthy things that little ones will do if they be Sulphur patients. The child is subject to catarrhal discharges from the nose, the eyes and from other parts, and he often eats the discharge from the nose. Now, that is peculiar, because offensive odors are the things that the Sulphur patient loathes. He is oversensitive to filthy odors, but filthy substances themselves he will eat and swallow. He becomes nauseated even from the odor of his own body and of his own breath.

The odor of the stool is so offensive that it will follow him around all day. He thinks he can smell it. Because of his sensitiveness to odors he is more cleanly about his bowels than anything else. It is an exaggerated sense of smell. He is always imagining and hunting for offensive odors. He has commonly such a strong imagination that he smells the things which he has only in memory.

The Sulphur patient has filthiness throughout. He is the victim of filthy odors. He has a filthy breath, he has an intensely foetid stool; he has filthy smelling genitals, which can be smelled in the room in spite of his clothing, and he himself smells them. The discharges are always more or less foetid, having strong, offensive odors. In spite of constant washing the axillae give out a pungent odor, and at times the whole body gives off an odor like that coming from the axillae.

Discharges: The discharges of Sulphur from every part of the body, besides being offensive, are excoriating. The Sulfur patient is afflicted with catarrhs of all mucous membranes, and the catarrhal discharges everywhere excoriate him. Often with the coryza the discharge excoriates the lips and the nose.

At times the fluid that remains in the nose smarts like fire, and when it comes in contact with the child's lip it burns, so acrid is it; almost like the condition under Sulphuricum acid, so red will be the parts that are touched by it.

There is copious leucorrhoea that excoriates the genitals. The thin feces cause burning and rawness around the anus. In women if a drop of urine remains about the genitals it will burn; very often it is not sufficient to wipe it away, it must be washed away to relieve the smarting.

In children we find excoriation about the anus and between the buttocks; the whole length of the fissure is red, raw and inflamed from the stool. From this tendency a keynote has been constructed, and not a bad one either, all the fluids burn the parts over which they pass," which is the same as saying that the fluids are acrid and cause smarting. This is true everywhere in Sulphur.

Skin: The Sulphur patient has all sorts of eruptions.

There are vesicular, pustular, furuncular, scaly eruptions, all attended with much itching, and some of them with discharge and suppuration. The skin, even without any eruption, itches much, itches from the warmth of the bed and from wearing woolen clothing.

Many times the Sulphur patient cannot wear anything except silk or cotton. The warmth of the room will drive him to despair if he cannot get at the itching part to scratch it. After scratching there is burning with relief of the itching. After scratching or after getting into the warmth of the bed great white welts come out all over the body, with much itching, and these he keeps on scratching until the skin becomes raw, or until it burns, and then comes a relief of the itching.

This process goes on continuously; dreadful itching at right in bed, and in the morning when he wakes up he starts in again and the eruptions itch and ooze. Crops of boils and little boil-like eruptions come out and this makes it useful in impetigo.

This remedy is useful in suppurations. It establishes all sorts of suppurating cavities, small abscesses and large abscesses; abscesses beneath the skin, in the cellular tissues and in internal organs. The suppurative tendency is very marked in Sulphur. The glands become inflamed and the inflammation goes on to suppuration.

Burning: Wherever there is a Sulphur complaint you will find burning.

Every part burns; burning where there is congestion; burning of the skin or a sensation of heat in the skin; burning here and there in spots; burning in the glands, in the stomach, in the lungs; burning in the bowels, in the rectum; burning and smarting in the hemorrhoids; burning when passing urine, or a sensation of heat in the bladder. There is heat here and there, but when the patient describes something especially typical of Sulphur she says:

"Burning of the soles of the feet in the palms of the hands, and on the top of the head."

Burning of the soles of the feet will very often be noticed after the patient becomes warm in bed. The Sulphur patient has so much heat and burning of the soles at night in bed that he puts the feet out from, beneath the clothes, sleeps with the feet outside the covering. The soles and palms of the Sulphur patient when examined present a thick skin which burns on becoming warm in bed.

Many complaints come on from becoming warm in bed. The Sulphur patient cannot stand heat and cannot stand cold, though there is a strong craving for the open air. He wants an even temperature; he is disturbed if the temperature changes much.

So far as his breathing is concerned, when he has much distress he wants the doors and windows open. The body, however, he is frequently forced to have covered, but if he is warmly clad he is bothered with the itching and burning of the skin.

Time: As to time aggravations, nightly complaints are a feature.

Headaches begin after evening meal and increase into the night; he cannot get to sleep because of the pain. There is nightly aching and nightly thirst; nightly distress and symptoms of the skin coming on after becoming warm in bed.

"Intermittent periodic neuralgia, worse every 24 hours, generally at 12 A.M. or 12 P.M."

Midday is another time of aggravation of the Sulphur complaints. It has chills at noon, fevers increase at noon, increase of the mental symptoms at noon, headache worse at noon. Complaints that come once a week, a seven-day aggravation, is another peculiar condition of Sulphur.

Diarrhea: it is a common feature for a Sulphur patient to have a peculiar kind of diarrhoea which has been long known as "a Sulphur diarrhoea," though many other remedies have a similar condition, viz.: diarrhoea coming on early in the morning.

The Sulphur diarrhoea belongs to the time between midnight and morning, but more commonly the time that he begins to think about rising.

The diarrhea drives him out of bed.

It is generally thin, watery; there is not much gushing, and it is not very copious, sometimes quite scanty, sometimes yellow faecal. After this morning stool he has, in many cases, no further trouble till next morning.

There are many people who go on year after year with, this urging to stool driving out of bed in the morning. The patient suffers from pain, griping, uneasiness, and burning soreness through the bowels. The stool burns while it is passing, and all parts that it comes in contact with are made sore and raw, and there is much chafing.

Desires and aversions: The Sulphur patient is very thirsty.

He is always drinking water. He wants much water.

He also speaks of a hungry feeling, a desire for food, but when he comes to the table, he loathes the food, turns away from it, does not want it.

He eats almost nothing, takes only the simplest and lightest things. There is a craving for stimulants, for alcohol, and an aversion to milk and meat; these latter make him sick and he loathes them.

One of the old men invented out of these things the keynote "drinks much and eats little."

This is true under Sulphur, but many other remedies have the same thing. As to the use of keynotes I would impress on you that it is well to gather together all the symptoms with their associations. It will not do to place much dependence on one little symptom, or even on two or three little symptoms. The symptoms of the whole case must be considered and then, if the keynotes and characteristics and everything else cause the remedy to be well rounded out and full, and to look like the whole patient, only then is it suitable.

There is emptiness occurring at 11 o'clock in the forenoon. If there is any time in the whole twenty-four hours that he feels hungry it is at 11 o'clock. It seems as though he cannot wait for his dinner.

There is this also about the Sulphur patient: he is very hungry about his customary mealtimes and, if the meal is delayed, he becomes weak and nauseated. Those that are accustomed to eat at about 12 o'clock will have that all-gone hungry feeling at 11 A.M. Those accustomed to eat about 1 or 1-30 will have it about 12 o'clock. The all-gone sensation is about one hour before the accustomed time of eating with many people.

In a sort of condensed way a strong Sulphur group is this: an all gone hungry feeling in the stomach at 11 A.M., burning of the soles and heat in the top of the head.

These three things have been looked upon as a sine qua non of Sulphur, but they are scarcely the beginning of Sulphur.

Skin: There is an unhealthy condition of the skin in Sulphur aside from the eruptions.

The skin will not heal. Small wounds continue to suppurate; abscesses formed under the skin become little discharging cavities with fistulous openings, and these leak and discharge for a long time.

Sulphur produces an infiltration in inflamed parts, so that they become indurated and these indurations last for years. When the inflammation is in a vital organ, like the lungs, this infiltration cannot always be endured; it leaves infiltrations after pneumonia called hepatization.

Sulphur produces this same tendency in inflamed parts throughout the body and hence its great use in hepatization.

Sulphur is a very useful remedy when the patient does not react after a prolonged disease, because of a condition in the economy, a psoric condition. When a patient is drawing near the end of an acute disease he becomes weak and prostrated. The inflammatory state ends in suppuration and infiltrations; the patient is in a state of weakness, much fatigued and prostrated, and has night sweats.

He does not convalesce after a typhoid or other acute disease. There is slow repair and a slow, tired economy, and order is not restored after the acute disease. Sulphur often becomes very useful in such conditions. Old drunkards become debilitated and have violent craving for alcohol; they cannot let liquor alone. They crave strong and pungent things, want nothing to eat, but want cold water and alcoholic drinks, They go on drinking till greatly exhausted and then their complaints come on. Sulphur will for a while take away this craving for drink and build him up.

The tissues seem to take on weakness, so that very little pressure causes soreness, sometimes inflammation and suppuration. Bed sores come on easily in a Sulphur patient, as there is feeble circulation. Induration from pressure is also a strong feature.

Sulphur has corns from pressure, callosities from pressure. These affections come easily. If a shoe presses anywhere on the skin a great corn or bunion develops. Where the teeth come in contact with the tongue and other parts of the buccal cavity nodules form and these little nodules in course of time commence to ulcerate.

It is a slow process with burning and stinging. They may go into cancerous affections. They may be postponed for a long time and afterwards take on a state of malignancy. Cancer is an outgrowth of a state in the body, and that state may come on from a succession of states. It is not one continuous condition, but the malignant state may follow the benign. Sulphur removes these states when the symptoms agree.

We notice a marked evidence of disturbance of the veins under Sulphur. It is a venous remedy, has much vein trouble. The veins seem to be relaxed and there is sluggish circulation. There is a flushed appearance of the face here and there from slight irritation, from the weather, from irritation of the clothing.

Tumefaction of the face. Sulphur has varicose veins; most marked of these are hoemorrhoidal veins, which are enlarged and burn and sting. Varices of the extremities. The veins even ulcerate, rupture and bleed. When going out of a cold into a warm atmosphere the patient suffers from enlarged veins, from puffiness of the hands and feet, from a sense of fullness throughout the body.

The Sulphur patient emaciates, and a peculiar feature is the emaciation of the limbs with distended abdomen. The abdomen is tumid, with rumbling, burning and soreness, and with the distended abdomen there is emaciation of all other parts. The muscles of the neck, back thorax and limbs wither, and the muscles of the abdomen are also wasted, but there is much distension of the abdomen itself. This condition of affairs is found in marasmus.

You will find a similar state under Calcarea; and, in women needing Calcarea, you will notice great enlargement, distension and hardness of the abdomen with shriveling of all other parts of the body.

Under Sulphur there are flashes of heat to the face and head, like those which women have at the climacteric period. The flash of heat in Sulphur begins somewhere in the heart region, generally said to be in the chest, and it feels as if, inside the body, a glow of heat is rising to the face. The face is red, hot and flushed, and finally the heat ends in sweat.

Flashes of heat with sweat and red face; the head is in a glow. Sometimes the patient will describe a feeling as if hot steam were inside the body and gradually rising up, and then she breaks out in a sweat. At times you will see a woman having little shiverings followed by flashes of heat and red splotches in the face, and then she fans vigorously; cannot fan fast enough, and she wants the doors and windows open.

Such is Sulphur as well as Lachesis and many others. When the flashes begin in the chest, about the heart, it is more like Sulphur, but when in the back or in the stomach it is more like Phosphorus.

Among other general aggravations we have an aggravation from standing in Sulphur. All complaints are made worse by standing for a length of time. Standing is the most difficult position for a Sulphur patient, and there is an aggravation of the confusion of mind, dizziness, the stomach and abdominal symptoms, and a sense of enlargement and fullness of the veins and dragging down in the pelvis in women, from standing. The patient must sit down or keep moving, if on her feet. She can walk fairly well, but is worse when standing quiet.

An aggravation after sleep fits into many of the complaints of Sulphur, but especially those of the mind and sensorium. Most of the, complaints of Sulphur are also worse after eating.

The Sulphur patient is aggravated from, bathing. He dreads a bath. He does not bathe himself and from his state in general he belongs to "the great unwashed." He cannot take a bath without catching "cold."

Children's complaints. Dirty-faced, dirty-skinned little urchins, who are subject to nightly attacks of delirium, who suffer much from, pains in the head, who bad brain troubles, who are threatened with hydrocephalus, who had meningitis, need Sulphur.

Sulphur will clear up the constitutional state when remedies have failed to reach the whole case because they are not deep enough. If the infant does not develop properly, if the bones do not grow, and there is slow closing of the fontanelles, Calcarea carbonica may be the remedy and Sulphur is next in importance for such slow growth.

You would not suppose that the Sulphur patient is so nervous as he is, but he is full of excitement; easily startled by noise, wakens, from sleep in a start as if he had heard a cannon report or seen a "spook."

The Sulphur patient is the victim of much trouble in his sleep. He is very sleepy in the fore part of the night, at times sleeping till 3 A.M., but from that time on he has restless sleep, or does not sleep at all. He dreads daylight, wants to go to sleep again, and when he does sleep he can hardly be aroused, and wants to sleep late in the morning. That is the time he gets his best rest and his soundest sleep. He is much disturbed by dreadful dreams and nightmare.

When the symptoms agree, Sulphur will be found a curative medicine in erysipelas. For erysipelas as a name we have no remedy, but when the patient has erysipelas and his symptoms conform to those of Sulphur, you can cure him with Sulphur.

If you bear that distinction in mind you will be able to see what Homoeopathy means; it treats the patient and not the name that the sickness goes by.

The Sulphur patient is annoyed throughout his economy with surgings of blood here and there surging, with fullness of the head, which we have heretofore described as flashes of heat. It has marked febrile conditions and can be used in acute diseases.

It is one of the natural complements of Aconite, and when Aconite is suitable to the acute exacerbations and removes them, very often Sulphur corresponds to the constitutional state of the patient.

Sulphur is suitable in the most troublesome "scrofulous" complaints in broken-down constitutions and defective assimilation. It has deep seated, ragged ulcers on the lower extremities, indolent ulcers, ulcers that will not granulate. They burn, and the little moisture that oozes out burns the parts round about. It is indicated often in varicose ulcers that bleed easily and burn much.

In old cases of gout, Sulphur is a useful remedy. It is a deep-acting remedy, and in most instances it will keep the gout upon the extremities, as its tendency is outward from centre to circumference.

Like Lycopodium and Calcarea, when suitably administered in old gouty conditions, not where there is much organic change present, it will keep the rheumatic state in the joints and extremities.

Sulphur, like Silicea, is a dangerous medicine to give where there is structural disease in organs that are vital, especially in the lungs.

Sulphur will often heal old fistulous pipes and turn old abscesses into a normal state, so that healthy pus will follow, when it is indicated by the symptoms. It will open abscesses that are very slow, doing nothing, it will reduce inflamed glands that are indurated and about to suppurate, when the symptoms agree.

But it is a dangerous medicine to administer in advanced cases of phthisis, and, if given, it should not be prescribed in. the highest potencies. If there are symptoms that are very painful, and you think that Sulphur must be administered, go to the 30 th or 200 th potency.

Do not undertake to stop the night sweats that come in the advanced stages, even if Sulphur seems to be indicated by the symptoms; the fact is, it is not indicated. A remedy that is dangerous in any case ought not to be considered as indicated, even though the symptoms are similar.

In old cases of syphilis, when, the psoric state is uppermost, Sulphur may be needed. Sulphur is rarely indicated when the syphilitic symptoms are uppermost, but when these have been suppressed by Mercury and the disease is merely held in abeyance, Sulphur will antidote the Mercury and allow the symptoms to develop and the original condition to come back in order to be seen.

The great mischief done by allopaths is due to the fact that they want to cover up everything that is in the economy; they act as if ashamed of everything in the human race; whereas Homoeopathy endeavors to reveal everything in the human rue and to antidote those drugs that cover up, and to free those diseases that are held down.

It is true that many patients will not have Homoeopathy because they do not want their syphilitic eruption brought to view they do not want the evidences of their indiscretion brought to light but Homoeopathy endeavors to do that. Conditions that are in the economy will come out under proper homoeopathic treatment. Sulphur brings complaints to the surface, so that they can be seen. It is a general broad antidote.

Suppressions: It is a medicine often called for in the suppression of eruptions from cold and from drugs, and even from Sulphur. It is a great medicine to develop these things which have been covered tip, hence you will see Sulphur in all the lists of remedies useful for suppressed eruptions or for anything suppressed by drugs.

Even when acute eruptions have been suppressed, Sulphur becomes a valuable remedy. In suppressed gonorrhoea Sulphur is often the remedy to start up the discharge and re-establish the conditions that have been caused to disappear. Symptoms that have been suppressed must return or a cure is not possible.

Sulphur has been the remedy from the beginning of its history, from the time of Hahnemann, and on his recommendation, to be thought of when there is a paucity of symptoms to prescribe on, a latent condition of the symptoms due to psora. In this state it has been administered with so much benefit that the routine prescriber has learned the fact.

When apparently (superficially) well-indicated remedies fail to hold a patient, and symptoms cannot be found for a better remedy, it is true that Sulphur takes a deep hold of the economy and remedies act better after it.

This is well established from experience. You will find at times when you have given a remedy which seems well indicated, that it does not hold the case, and then you give the next best indicated remedy, and then the next, with the same result.

You will begin to wonder why this is, but you will see that, although the case does not call clearly for Sulphur, yet on its administration it so closely conforms to the underlying condition (and psora is so often the underlying condition) that it makes the remedies act better.

This is an observation that has been confirmed since the time of Hahnemann by all the old men.

Such things are only necessary when there is a paucity of symptoms, where after much study it is necessary to resort to what, seem the best measures, measures, justifiable to a certain extent, based upon observation and upon a knowledge of the conditions underlying the constitution of the whole race.

We know that underlying these cases with few symptoms there is a latent condition, and that it is either psora, syphilis or sycosis. If it were known to be syphilis we would select the head of the class of remedies looking like syphilis.

If known to be sycosis, we would select the head of the class of remedies looking like sycosis. Sulphur stands at the head of the list of remedies looking like the underlying psora; and so, if the underlying constitution is known to be psoric, and it is a masked case, Sulphur will open up the latent cause, and, even if it does not act on a positively curative basis, it is true that a better representation of the symptoms comes up. And as Sulphur is to psora, so is Mercurius to syphilis, and Thuja to sycosis.

In the coal regions of Pennsylvania, those who work in the mines and those living in the vicinity of the mines often need Sulphur. We know that the coal is not made up of Sulphur; there is a good deal in it besides; but those who handle the coal often need Sulphur. Persons who are always grinding kaolin and the various products that are used in the manufacture of china, and the workers among stone, especially require Calcarea and Silicea, but those who work in the coal mines often need Sulphur.

The patients look like Sulphur patients; they have the aspect, and even when their symptoms are localized and call for other remedies, you will get no good action from these remedies until you give them a dose of Sulphur, after which they go on improving.

Some believe this is due to the fact that there is so much Sulphur in the coal. We may theorize about these things as much as we have a mind to, but we do not want to fall into the habit of antidoting the lower potencies with the high.

Only use that method as a dernier ressort. When there are no symptoms to indicate the remedy, then it is time for us to experiment, and then it is justifiable only when it is carried on by a man of the right sort, because such a man keeps within the limit. He knows how to give his remedy. Such a man is guided by the symptoms in each case so far as symptoms speak out.

In inflammatory conditions a purplish appearance of the inflamed parts, a venous engorgement, is seen under Sulphur.

Measles, when they come out with that purplish color, very often require Sulphur.

Sulphur is a great remedy in measles.

The routinist can do pretty well in this disease with Pulsatilla and Sulphur, occasionally requiring Aconite and Euphrasia. Especially will Sulphur modify the case when the skin is dusky and the measles do not come out. This purplish color may be seen anywhere, in the erysipelas, in the sore throat, often on the forearms, legs and face.

The dreadful effects of vaccination are often cured by Sulphur. In this it competes with Thuja and Malandrinum.

Mind: In the mental state, which gives out the real man, shows forth the real interior nature, we see that Sulphur vitiates his affections, driving him to a most marked state of selfishness.

He has no thought of anybody's wishes or desires but his own. Everything that he contemplates is for the benefit of himself. This selfishness runs through the Sulphur patient. There is absence of gratitude.

Philosophical mania is also a prominent feature. Monomania over the study of strange and abstract things, occult things; things that are beyond knowledge; studying different things without any basis to figure upon; dwelling upon strange and peculiar things.

Sulphur has cured this consecutive tracing one thing to another as to first cause. It has cured a patient who did nothing but meditate as to what caused this and that and the other thing, finally tracing things back to Divine Providence, and then asking:

"Who made God?"

She would sit in a corner counting pins and wonder, pondering over the insolvable question of "Who made God?"

One woman could never see any handiwork of man without asking who made it. She could never be contented until she found out the man who made it, and then she wanted to know who his father was; she would sit down and wonder who he was, whether he was an Irishman, and so on.

That is a feature of Sulphur. It is that kind of reasoning without any hope of discovery, without any possible answer. It is not that kind of philosophy which has a basis and which can be followed up, reasoning in a series, reasoning on things that are true, but a fanatical kind of philosophy that has no basis, wearing oneself out.

Sulphur has an aversion to follow up things in an orderly fashion, an aversion to real work, an aversion to systematic work. The Sulphur patient is a sort of inventive genius. When he gets an idea in his mind he is unable to get rid of it.

He follows it and follows it until finally accidentally be drops into some thing, and many times that is how things are invented. Such is a Sulphur patient. He is often ignorant but imagines himself to be a great man; he despises education and despises literary men and their accomplishments, and he wonders why it is everyone cannot see that he is above education. Again, this patient takes on religious melancholy, not meditating upon the rational religion, but on foolish ideas about himself. He prays constantly and uninterruptedly, is always in his room, moaning with despair. He thinks he has sinned away his day of grace.

A patient needing Sulphur is often in a state of dullness and confusion of mind, with inability to collect the thoughts and ideas; lack of concentration. He will sit and meditate on no one thing continuously, making no effort to concentrate his mind upon anything. He wakes up in the morning with dullness of mind and fullness in the head and vertigo. Vertigo in the open air. In the open air comes on coryza with this fullness in the head and dullness, so that there is a confusion of the mind.

In the books there is an expression that has been extensively used.

"Foolish happiness and pride; thinks himself in possession of beautiful things; even rags seem beautiful."

Such a state has been present in lunatics, and in persons who were not lunatics in any other way except on that one idea.

The Sulphur patient has an aversion to business. He will sit around and do nothing, and let his wife take in washing and "work her fingernails off" taking care of him; he thinks that is all she is good for.

A state of refinement seems to have gone out of the Sulphur patient. Sulphur is the very opposite of all things fastidious. Arsenicum is the typical fastidious patient, and these two remedies are the extremes of each other. Arsenicum wants his clothing neat and clean, wants everything hung up well upon the pegs, wants all the pictures hung up properly upon the wall, wants everything neat and nice; and hence the Arsenicum patient has been called "the gold-headed-cane patient," because of his neatness, fastidiousness and cleanliness.

The very opposite of all that is the Sulphur patient.

"Indisposed to everything, work, pleasure, talking or motion; indolence of mind and body."

"Satiety of life; longing for indolence of mind and body.''

"Satiety of life; longing for death."

"Too lazy to rouse himself up, and too unhappy to live."

"Dread of being washed (in children)."

Yes, they will cry lustily if they have to be washed. The Sulphur patient dreads water and takes cold from bathing.

As to its relationship, Sulphur should not be given immediately before Lycopodium. It belongs to a rotating group, Sulphur, Calcarea, Lycopodium.

First Sulphur, then Calcarea and then Lycopodium, and then Sulphur again, as it follows Lycopodium well. Sulphur and Arsenicum are also related.

You will very often treat a case with Sulphur for a while and then need to give Arsenicum for some time, and then back to Sulphur. Sulphur follows most of the acute remedies well.

The Sulphur patient is troubled is with much dizziness. When he goes into the open air or when he stands any length of time, he becomes dizzy. On rising in the morning his head feels stupid, and on getting on his feet he is dizzy.

He feels stupid and tired, and not rested by his sleep, and "things go round." It takes some time to establish an equilibrium. He is slow in gathering himself together after sleep. Here we see the aggravation from sleep and from standing.

Head: The head furnishes many symptoms.

The Sulphur patient is subject to periodical sick headaches; congestive headaches, a sensation of great congestion with stupefaction, attended with nausea and vomiting. Sick headache once a week or every two weeks, the characteristic seven-day aggravation.

Most headaches coming on Sunday in working men are cured by Sulphur. You can figure this out. Sunday is the only day he does not work, and he sleeps late in the morning and gets up with a headache that involves the whole head, with dullness and congestion. Being busy and active prevents the headache during the week.

Others have periodical headaches every seven to ten days, with nausea and vomiting of bile. Again he may have a headache lasting two or three days; a congestive headache. Headache with nausea and no vomiting or headache with vomiting of bile. The headache is aggravated by stooping, generally ameliorated in a warm room and by the application of warmth; aggravated from light, hence the desire to close the eyes and to go into dark room; aggravated by jarring, and after eating.

The whole head is sensitive and the eyes are red, and there is often lachrymation, with nausea and vomiting. Headaches at times in those who suffer constantly from great heat in the vertex; the top of the head is hot and burns and he wants cold cloths applied to the top of the head. These headaches associated with heat are often ameliorated by cold, but otherwise the head is ameliorated in a warm room.

The head feels stupid and sometimes he cannot think. Every motion aggravates and he is worse after eating and drinking, worse from taking cold drinks into the stomach and better from hot drinks. When the headaches are present the face is engorged; bright red face. Headaches in persons who have a red face, a dirty face or sallow, a venous stasis of the face; the eyes are engorged and the skin is engorged; the face is puffed and venous in appearance.

Sulphur is useful in persons who get up in the morning with headache, dizziness and red face; in persons who say they know they are going to have the headache some time during the day because the face feels very full and is red in the morning, and the eyes are red.

Before the headache comes on there is a flickering before the eyes, a flickering of color. Scintillations, stars, saw teeth, zig-zags are forewarnings of a headache. Some Sulphur headaches that I have known present a peculiar appearance before the eyes; a rhomboidal figure, obliquely placed, with saw teeth on the upper side and the body filled with spots.

Sometimes this figure is seen toward one side of the object looked at, sometimes on the other side, but it is seen equally distinct with both eyes at the same time.

These saw teeth are flashes of light, and the base of the figure grows increasingly darker until you get all the colors of the rainbow. Whenever he disorders his stomach he has this peculiar vision. Sometimes it comes in the morning after eating and sometimes at noon after eating. It comes also when be is hungry in the evening and delays his eating. These zig-zags come very often with that hungry all-gone feeling in the stomach.

We have the same state of affairs, similar appearance of zig-zags and flickerings in both Natrum muriaticum and Psorinum before the headache. They are warning of headaches. These zig-zags, flickerings, sparks, stars and irregular shapes appear before the eye periodically, and may last an hour or so. In the head there is much throbbing. Morning headaches and headaches coming on at noon. Headaches also, as mentioned before that begin after the -evening meal and increase into the night, hindering sleep.

Scalp: Upon the external head the itching is indescribable; constant itching, itching when warm in bed.

It is worse from the warmth of the bed and yet it is also worse from cold. itching eruptions; scaly, moist and dry eruptions; vesicles, pimples, pustules and boils; eruptions in general upon the scalp. Much dandruff in the hair, and loss of hair There is slow closing of the fontanelles.

"Humid, offensive eruption on top of the head, filled with pus, drying up into honey like scabs. Tinea capitis."

"Humid offensive eruption with thick pus, yellow crusts, bleeding and burning."

Hair dry, falling off, etc.

It has many symptoms, such as in olden times would be called scrofulous, but which we recognize as psoric. There is a tendency for every "cold" to settle in the eyes. Discharge of mucus and pus from the eyes. Ulceration and thickening of the eyelids, lids rolled outwards or inwards, loss of eyelashes; red and disturbed condition.

Eyes: Now, if we would say "complaints of the eyes in a Sulphur patient," it would cover all kinds of eye troubles.

Sulphur has extensive eye symptoms. Eye symptoms with eruptions on the face and scalp, with itching of the skin, especially when warm in bed. Catarrhal eye symptoms that are made worse from washing.

When not only the eyes are aggravated by bathing, but the patient himself is aggravated from bathing and he dreads to bathe, and he has itching which is made worse from the warmth of the bed, and is subject to chronic sick headaches and has heat on top of the head, with such concomitants his eyes symptoms, no matter what will be cured by Sulphur. Sulphur has cured, cataract and iritis, inflammatory conditions and opacities, and all sorts of "hallucinations of sight" (coming with headaches)

"Flickering before the eyes" (as described)

"small dark specks; dark points and spots; black flies seem to float not far from the eyes; gas or lamp light seems to be surrounded with a halo," etc.

There are so many of these peculiar images before the eyes, but all have the Sulphur constitution.

"Burning heat in the eyes, painful smarting."

Every "cold" settles in the eyes, i. e,, the eye symptoms, when present, are increased and, when he has no eye symptoms, these are brought on from every "cold."

Ears: The ears are subject to catarrh.

You have learned in the generals, that the catarrhal state is a very strong feature of Sulphur. No mucous membrane of the body escapes, all have catarrhal discharges, copious, sometimes purulent, sometimes bloody. The eyes and ears are no exception.

The catarrhal state goes on in a patient until deafness follows. Thickening of the mucous membrane and of the drum. All sorts of strange noises in the ear until the hearing is lost. After structural changes have taken place and deafness comes on even if there is no cure for the deafness, you may cure the patient.

When a patient wants to know if he can be cured of his deafness you can never tell him. Many of the troubles are in the middle ear, and as you cannot examine it, you do not know how much structural change has taken place. You can only say that if the patient can be sufficiently cured then it can be ascertained. If the structural changes are not very great they will disappear after the patient is cured.

If the internal parts are destroyed, if there is a dry, atrophic catarrh of the middle ear, you can hardly expect to restore that middle ear. It has been destroyed; the parts that are necessary for sensation no longer register sensation, because they have become atrophied. You can only talk to the patient about the prospects of curing him.

Do not entertain in your mind the idea of curing an organ. Keep that idea out of your mind as much as possible, and, when people want you to locate the disease in organs, keep quiet, because only the patient is sick.

Think as much as possible about the sick patient and as little as possible about the name or the pathological conditions of organs. So, when patients say,

"Doctor, can you cure my hearing?" answer them:

"First, you must be cured. The first and most important thing is to cure you."

Cure the patient and then it will be seen afterwards what can be done for the ear, for the hearing. That keeps your mind in proper form, keeps you in right relation to the patient. If you were all the time talking of the ear, the patient would worry your life out about his ear.

"When are you going to do something for ear? When am I going to hear?"

Start out with the understanding that the whole patient is to be treated. Remember the patient first, and let him understand that. The idea of a patient going to a specialist for diseases of the ears should be discouraged unless a homoeopathist is at hand.

It is a disease of the whole body that is to be treated. There is no such trouble as an ear trouble considered apart from the constitutional state of the patient himself. Sulphur has "frequent stoppages of the ears, especially when eating or blowing one's nose."

"Sounds in ears."

Inflarnmation of various kinds. Discharges from the ears in a Sulphur patient.

You see I have avoided saying that Sulphur is a remedy for the ears. Many times you will cure patients of these "local diseases" if you select remedies for the patients, when the local symptoms would never have led you to the remedy.

You would never have thought of Sulphur for the ear alone, or for the prolapsus of the uterus, yet the patient needs Sulphur, and, having given it, you are astonished to see how the organs are turned into order after the constitution of the patient has been made orderly.

Now and then pains that are located here and there in the body are prescribed at by the physician, and failure follows. He hunts a remedy through and through to find some particular kind of pain that resembles the pain which the patient has. You should treat the patient and not bother about trifling pains.

Leave it out if you want to, but get a remedy for the patient. If that pain is in the remedy well and good, but if not do not bother about it. Do not bother about the little symptoms. You may even leave out a most prominent keynote in treating the patient.

Sometimes that particular pain is the only symptom the patient wants cured, but if it is an old symptom, it will be the last thing to go away. Under such circumstances the patient will bother your life out wanting to know when that pain is going to be cured, but if you have knowledge of the matter you will not expect to relieve that pain the first time; if you do relieve it you know that you have made a mistake, for the later symptoms should all go away first.

It is sometimes necessary, in order to hold a patient, to say,

"That symptom must not be cured first, but these little symptoms that you do not care much about will go away first."

You will hold that patient for life simply because you have told the truth, simply because you have exhibited to her that you know. Such business is honestly acquired business.

Nose: The catarrhal affections of the nose are extremely troublesome in Sulphur.

"Smell before the nose as of an old catarrh," and so troublesome is the Sulphur nose, so troublesome is this catarrhal state that with odors he is made sick. He thinks he smells his own catarrh, and thinks others also smell it. The smell of this old catarrh, or of filthy things, keeps him nauseated. He is subject to coryzas; constant sneezing, stoppage of the nose. Under coryza we read "fluent like water trickling from the nose."

All the nasal discharges are acrid and burning.

This is a state in Sulphur. Every time he takes "cold," it brings on a coryza. He cannot take a bath, he cannot become overheated, he cannot get into a cold place and cannot overexert himself without getting this "cold in the nose."

Changes of the weather establish a new attack, I have observed in numbers of those old people who are in the habit of taking large quantities of Sulphur in the spring for boils, and as a spring cleanser, that for the rest of the year they suffer from coryza and the various complaints of Sulphur.

If you can hunt out some of these old Sulphur takers, you will have a very good picture of Sulphur, interesting for the homeopathic physician to look upon. He is also subject to nose-bleed, dry ulcers and scabs in the nose.

Face: I have quite sufficiently described the general aspect of the face in Sulphur, but we must especially remember the venous stasis, the dirty appearance, the red spots, the sickly look, the appearance of false plethora. It is a face that changes from pale to red, a pallid face that becomes easily disturbed, flushed from excitement, flushed in a warm room, flushed from slight stimulation, especially flushed in the morning. Eruptions upon the face.

Periodical neuralgias of the most violent character, especially on the right side of the face. Long and tedious right-sided neuralgias. Persistent neuralgias in those that live in a malarial climate, when the short-acting remedies given for the neuralgia, such as Belladonna and Nux vomica, have only for a short time mitigated the suffering. If upon studying the whole case you find he turns out to be a Sulphur patient, Sulphur will permanently cure the neuralgia.

Sulphur cures erysipelatous inflammation of the face. In Sulphur the erysipelas commences on the right side of the face and about the right ear, and there is considerable swelling of the right car, and it spreads slowly, moves with sluggishness and is unusually purple. The whole patient is an offensive, filthy patient; in spite of washing, his skin looks wrinkled, shriveled and like dried beef.

Sulphur is not so suitable in the cases that come on with rapidity and great violence, with vesicles and enormous blebs, but it suits those cases in which at first there is the appearance of a mottled dusky red spot on the face, and a little distance from it another spot and the another, and these, as it were, all run together, and after a week or so it develops into a sluggish erysipelatous state, and the veins seem to be distended, and he is passing into a state toward unconsciousness.

You will be astonished to see what Sulphur will do in such a case, which comes slowly as if there were a lack of vitality to develop it, a slow, sluggish, erysipelatous inflammation. Whereas, if it be Arsenicum, Apis or Rhus tox, it spreads with rapidity. Arsenicum and Apis burn like, fire and Rhus has blisters upon the erysipelatous patches.

The whole face in Sulphur is covered at times with patches of moist, scaly, itching, eczematous eruptions. Crusta lactea that involves the scalp and the cars, with moisture, thick yellow crusts, piling up, with much itching, which is worse when warm in bed. The child sleeps without covers. If there is itching in parts that are covered, when the parts become warm the itching increases. These eruptions are associated with eye diseases, catarrhal affections of the eyes and nose.

The Sulphur patient has thick incrustations upon the lips, scabby lips, chapped lips, cracks about the lips and corners of the mouth. The saliva oozes out of the mouth making red streaks. Eruptions with itching and burning about the lower part of the face. Herpetic eruptions about the mouth. All of these burn and become excoriated from the fluids of the mouth. Round about the under jaw there is swelling of the glands. Swelling and suppuration of the sub-maxillary glands swelling of the parotids. The glands of the neck are enlarged.

Teeth and Mouth: In the Sulphur constitution the teeth become loose; the gums settle away from the teeth and bleed and burn.

The teeth decay. There is a general unhealthy condition of the mouth and tongue. Foul taste and foul tongue. Ulceration of the mouth and burning in the ulcers. In the aphthae there is burning, stinging. White patches in the mouth.

Sulphur is a very useful remedy in sore mouth of nursing infants, and such as occurs in the mother during lactation. It has also deep-seated phagedenic ulcers that eat around the inner surface of the check. Peculiar little nodules form upon the tongue. and upon the sides of the mouth where the unhealthy teeth press.

When these nodules come along the edge of the tongue they are so painful that he cannot talk and cannot swallow. He must live on substances that he can take without having to move the tongue. Sometimes they involve the whole tongue, and have been called cancerous affections even when benign.

Throat: Sulphur is a wonderful medicine for chronic sore throat when the symptoms agree.

The old Sulphur patient suffers from a general catarrhal state, as has been said, and the throat symptoms are of that sort. There is a catarrhal state which goes on even to ulceration. The tonsil is enlarged, and of a purplish aspect lasting for weeks and months, a general sore and painfully sensitive condition of the throat; but it has also an acute sore throat. It is especially useful in inflammation of the tonsil with suppuration, when the aspect is purplish, venous, and not a bright red inflammation.

The purplish, dusky color is especially a Sulphur color. There is often burning in the throat, stitching, rawness, smarting, inflammation and difficult swallowing. It has cured diphtheria.

I have sufficiently covered appetite, desires and aversions under the generals. The Sulphur patients are commonly dyspeptics, patients who can digest almost nothing. They must live on the simplest forms of food in order to have any comfort at all; cannot digest anything like ordinary diet.

The stomach: is sensitive to touch with the all-gone hungry feeling before mealtimes.

The Sulphur patient cannot go long without eating; he becomes. faint and weak. Great heaviness in the stomach after eating but little, after eating meat, or after eating foods that require a healthy stomach to digest.

Then he becomes the victim of pain. He will describe the pains in his stomach as burning pains and great soreness; he has a morbid feeling in the stomach; smarting, and rawness in the stomach. He will describe this sensation as

"Pain in the stomach after eating.

Sensation of weight in the stomach after eating," etc.

The Sulphur stomach is a weak stomach, is slow in digesting. There is acid and bilious vomiting, as a result of the disordered stomach. Sour taste in the mouth from the welling up of acids from the stomach.

The liver: is a very troublesome organ.

There is enlargement and induration, with much painfulness, pressure and distress. With congestion of the liver, the stomach also takes on its usual symptoms, or, if present already, they are aggravated. The patient becomes jaundiced, with sensation of engorgement or fullness of the liver, dull aching in the liver.

He is subject to gall stones; tearing pains in the region of the gall duct, coming periodically, attended with much increase of his sallowness.

The Sulphur liver patient is the victim. of chronic sallowness, which increases and decreases. When this patient takes "cold" it settles in the liver; every "cold," every bath he takes, every change of weather, aggravates his liver symptoms, and when these are worse he has less of other troubles. It localizes itself in attacks of bilious vomiting, in attacks of "bilious headaches," as he calls them.

At times the stool is black as tar, at others it is green and thick, and there are times when the stool is white. These stools alternate and change about with the engorgement of his liver, and then he is subject to gall stones.

Abdomen: The Sulphur patient suffers from great distension of the abdomen; rolling in the abdomen; soreness in the abdomen.

He cannot stand because the abdominal viscera hang down so; they seem to be falling. There is rawness, soreness, distension and burning, with diarrhoea with chronic diarrhoea, and then this goes on to more serious trouble, towards tubercle in the abdomen. The mesenteric glands become in filtrated with tubercle. There is nightly itching with the eruptions upon the abdomen, the itching being worse when warm in bed. Shingles come out about the sides and seem inclined to encircle the body.

He is also a flatulent patient. There is much belching, much distension, much rumbling and passing of flatus. He has spells of colic without being flatulent; the wind is confined. Dreadful spells of colic, cutting, tearing pains relieved in no position; burning and smarting in the whole abdomen and soreness of the intestines.

Catarrh of the whole intestinal tract. That which he vomits is acid and smarts the mouth, and that which he passes by the anus is acrid and makes the parts raw. The liquid stool burns while it is passing, and there is much burning when passing moist flatus. He is often called to stool, but while sitting at stool he passes only a little fluid or a little moisture with flatus, and that fluid burns like coals of fire, and the anus becomes raw.

The stool may be thin faeces, yellow, watery, mucous, green, bloody, excoriating. The stool is offensive, often sickening, of a penetrating odor which permeates the room, and "the smell of the stool follows, him around, as if he had soiled himself."

The diarrhea comes on especially in the morning and it is commonly limited to the forenoon. It drives him out of bed in the morning; as soon as he wakes up and moves in bed, he feels the urging to stool and must make great haste, or he will lose it; it is with difficulty that he can hold it until he reaches the commode.

The morning is the typical time, but a diarrhea that comes on any time after midnight, from midnight till noon, may be a Sulphur diarrhea. Very seldom would you -expect to cure with Sulphur a diarrhea that is in the habit of coming on during the afternoon. Sulphur has some evening aggravations in diarrhea, but these are exceptions; it is the morning diarrhoea that we look to Sulphur to cure.

Sulphur is a wonderful remedy in cholera and in those cases of diarrhea that occur in cholera times, when the diarrhoea begins in the morning. It is also of great value in dysentery, when the stool is bloody mucus with constant straining. As in Mercurius he must sit long at stool because of a feeling as if he could not finish. Such is the typical Mercurius state: a slimy stool with the sensation as if he could not finish.

Sulphur often cures this state after Mercurius fails. It is the natural follower of Mercurius when the latter has been misunderstood and given. In dysentery, when this tenesmus is of the most violent character, when the stool is pure blood, when it is attended also with much urging to urinate.

Mercurius corrosivus gives the quickest relief. If the tenesmus is less violent, and there is not much straining to urinate, or it is altogether absent, Mercurius solubilis is the more natural remedy. These medicines run very close to Sulphur in dysentery, but are more commonly indicated than Sulphur. In Sulphur patients of course Sulphur will be the suitable remedy in dysentery.

He is subject to hemorrhoids, external and internal; great bunches that are sore and raw, burning and tender, and that bleed and smart with the liquid stool.

The urinary symptoms: and those of the bladder and male sexual organs, combine to give a very important group in Sulphur.

There is a catarrhal state of the bladder, constant urging to urinate and burning and smarting while urinating. The urine scalds the urethra while passing and the smarting is so great that it lasts a long time after urination. It is indicated in broken-down constitutions, in old inventors, in old philosophers who have been leading sedentary lives, who suffer from enlarged prostate. burning in the urethra during and after the flow of urine, and a urethral discharge not unlike gonorrhoea, but really a chronic catarrhal state.

Mucus in the urine, sometimes a pus. In old cases of gleet, in old broken-down patients, when the ordinary gonorrhoea remedies, and the remedies especially fitted to the discharge itself, only palliate; when the patient himself is a Sulphur patient.

Such a patient has had a gonorrhoea and has been treated by remedies adapted to the new appearance, to the discharge itself, but a catarrhal state of the urethra follows, with burning in the urethra, swelling of the meatus, a red, swollen, pouty condition of the meatus, and only a drop collects, just enough to soil the linen, and this keeps up week after week, and sometimes for years; he will be cured of this discharge by allowing potentized Sulphur to act long enough.

Sulphur has cured patients with sugar in the urine, in the early stage of diabetes. Sulphur cures involuntary urination during sleep. It cures troubles brought on by taking "cold." Every "cold" in some patients settles in the bladder. This is like Dulcamara, and when Dulcamara will no longer hold, or when it has been suitable in earlier stages, Sulphur follows it well. Continuous smarting of urine and frequent urging; burning, stinging, smarting in the urethra for a long time after micturition.

Genitals: On the genitals there are many eruptions.

Itching of the genitals, worse from warmth of the bed; much sweat about the genitals; coldness of the genitals. In the male, impotency; the sexual desire is fairly strong, but he is unable to secure suitable erections; or there is discharge of semen before intromission, or too soon after intromission. There is an inflammatory condition around the glans and foreskin. Herpetic eruptions under the foreskin, itching and burning. This patient has much annoyance from itching eruptions on the genitals.

The prepuce becomes narrow and cannot be drawn back inflammatory phimosis; thickening or restriction of the prepuce. Inflammatory phimosis can be cured by remedies, if the phimosis depends upon some, trouble that is in itself curable. Congenital phimosis cannot be cured by remedies. The genitals are extremely offensive both to the patient and to the examining physician. The patient is likely to be very uncleanly; he does not bathe himself, and the genitals accumulate their natural filth. Discharge of prostatic fluid when at stool.

Under female sexual, organs we have sterility. We have irregularity in the menstrual flow, menstrual flow suppressed from the slightest disturbance. Haemorrhage in connection with the menstrual flow; uterine haemorrhage; prolonged uterine haemorrhage.

In an abortion you may have selected Belladonna, which war, suitable while the woman was aborting, and it may have overcome the present state; or you may have selected Apis or Sabina, which was suitable for the early state, and it either postpones or checks the haemorrhage for the time or hurries the expulsion of the foetus; but the haemorrhage starts in again and with its return we have prolonged tribulation. In many of these cases we can do nothing until we put the patient on Sulphur.

If the symptoms are masked, Sulphur stands very high. When Belladonna has been given you will often have to follow it with Sulphur. Sabina, which has the most violent gushing haemorrhage in abortions, very commonly needs to be followed by Sulphur. In such hemorrhagic affections, however i.e, in a prolonged recurring haemorrhage, a chronic condition, not in the first or most exciting time, not in the time of the earliest gushing, there are two very frequently indicated remedies, M.: Sulphur and Psorinum.

The flow keeps coming back in spite of ordinary remedies, and in spite of remedies selected upon the group of symptoms related to the pelvis. In many instances we go to a haemorrhage and the pelvic symptoms are prominent and all other symptoms clouded; there is a gushing flow, the blood is hot, etc., and there are only a few symptoms; but the next time you see the woman she is quiet enough to give other symptoms, and in the course of a few days more symptoms come out, as the hemorrhagic state is an outcome of the chronic condition.

This is unlike measles. You do not have to look into the chronic state until the measles or scarlet fever or small-pox is finished; these are acute miasms. But the haemorrhage is a part of her constitutional state; it is not a miasm; and hence when it is violent, calling for a remedy, probably the best adapted will he the short acting remedies, such as Belladonna or even Aconite; but then look into the constitutional state for it is likely some remedy will have to follow the Aconite or the Belladonna, and commonly it is Sulphur; the acute remedy being suitable to the violent action and then followed by its complementary medicine.

Women needing Sulphur are full of hot flashes, such as they are likely to have at the climateric period, and here it competes with Lachesis and Sepia. Sulphur and Sepia are suitable in the most violent cases of dysmenorrhoea in girls and even in those of advanced age.

Most violent cases that have existed a long time, since the beginning of menstruation, in women who always needed Sulphur. If you select a remedy merely on the kind of pain, on the sensitiveness of the uterus, on the appearance of the flow, i.e., on the pelvic symptoms, you will make a failure. You must treat the patient, even if the pelvic symptoms do not fall under the generals; when the generals agree Sulphur will cure dysmenorrhoea even though you cannot fit it to the pelvic symptoms. The generals always precede and rule in every case.

Sulphur has violent burning in the vagina. Troublesome itching of the vulva. Great offensiveness from the genitals. Perspiration copious and foetid coming from about the genitals, down the inside of the thighs and up over the abdomen.

She is so offensive that the odors nauseate her, and this general state is true, it is not the imagination. Remember the over-sensitiveness to odors. Leucorrhoea copious, offensive, burning, sticky; it may be whitish or yellow; it is offensive, acrid, and causes itching about the part and excoriation.

There is much nausea during gestation, or only during the early period of gestation. In those women needing Sulphur, it will stop the nausea, and they will go into labor easily, with few protracted pains; they will go through their labor with only the contractions, and these comparatively painless. The only pains in such cases will be those from the pressure of the child's head. Labor is painful we know, but it is comparatively easy when the woman is upon a suitable remedy. Sulphur is indicated then in women who have suffered from the most dreadful agony in confinement; prolonged labor. Troublesome after-pains. Suitable also in swelling of the mammary glands.

Then we have septicemic conditions, with purulent lochia or suppression of the lochia. You may go to a case in which, on the third day, there has been a chill, the lochia has been suppressed, the woman has a high temperature and is covered from head to foot with sweat. As you put your hand under the covers you feel steam come up from the body so that you want to take your hand away, it is so hot. She is dazed and is sensitive over the whole abdomen.

You know now the meaning of the suppression of the lochia; you have a puerperal fever on hand. Study closely for Sulphur instead of hunting around among Aconite, Bryonia, Belladonna, Opium, etc. With these you will make a total failure in most instances, but Sulphur fits into just such a state and has cured many cases of puerperal fever.

If it is but a milk fever or mammary indisposition and the chill is only acute, then your short-acting remedies will do very well and even Aconite has been useful, but when it is a case of septicemia, Sulphur goes to the very root of it. When the feet burp, when there is a hungry feeling in the stomach, the night aggravation with sinking and exhaustion, and when throughout the body there is a sensation of steam rising or hot flashes, one after another, you must give Sulphur.

Now, on the other hand, if in such a case, with the hot sweat and other general features, you have one rigor following another in rapid succession and no end to them, you cannot get out of that case without Lycopodium, which goes as deeply into the case as Sulphur. When there is a continuous intermingling of little chillinesses and little quiverings throughout the body and the pulse has lost its proper relationship to the temperature, Pyrogen must be administered. If there is a purplish appearance of the body, cold sweat all over, if there are remittent or intermit-tent chills, with thirst during -the chill, and at no other time, and the face is red during the chill, you must give Ferrum, as no other remedy looks just like that.

When one side of the body is hot and the other side is cold and you find the woman in a tearful state, trembling with fear, nervous excitement and restlessness, give Pulsatilla, which also has a septic state and is sufficient to overcome the septic condition.

Sulphur is suitable in surgical fever when it takes this form of flashes of heat and steaming sweat. In these deep-seated septic states, somewhere from beginning to end, Sulphur will most likely be wanted. You may see in the earlier stages of that septic state a number of Bryonia symptoms, but Bryonia cannot take hold of that case. Remember that in a septic state you want to get ahead of it in the first twenty-four hours; you do not want to let it run on, and if Bryonia has only mitigated it in its beginning then it is too late for Sulphur.

Go to Sulphur at once. Now, another thing, even if you have made a mistake in giving Sulphur and you find it does not take hold of the case, it always simplifies it, does good and never spoils it. It gives you a good basis to begin on. It goes to the bottom and simplifies the matter, and, if you have mental and nervous symptoms left still you have overcome that violent septic state which must be met at once, and the remaining symptoms in many instances are simple. Sulphur is a general remedy to begin with in those cases where the symptoms are not perfectly clear for another.

Respiration: This remedy is full of difficult breathing, shortness of breath from very little exertion, copious sweat, so exhausted; asthmatic breathing and much rattling in the chest.

Every time he gets "cold" it settles in the chest or in the nose. In both these instances the catarrhal state hangs on and holds a long time; it seems never to be finished, always remains as a catarrhal state.

"Every cold he takes ends in asthma," calls for Dulcamara, but very often the fag end of that attack will remain and the physician has to give a deep-acting remedy. After Dulcamara has done all it can do, Sulphur comes in as its complementary remedy. Calcarea carb. has a similar relationship to Dulcamara.

The nose, the inner chest and lungs furnish us localities for much trouble. The patient has had pneumonia and it has gone on to the period of infiltration; you have taken the case in this advanced stage after Bryonia has overcome the threatening features, and now when the patient should rally he does not rally; he perspires all over, is tired and has a strange and singular consciousness that "there is something wrong in there; a load in there;" "difficult breathing; flashes of heat and yet not much fever; sometimes coldness alternating with flashes of heat. I have often heard them say, "

There is a great load in there, doctor. I cannot get rid of it."

Upon close examination you find there is hepatization and now comes the time for such remedies as Phosphorus, Lycopodium and Sulphur, and Sulphur leads them all.

When Bryonia has been sufficient for the earlier symptoms, or when Aconite has cleared them up, there has been too much for these remedies to relieve, then hepatization comes on. If this is confined to only a small area it will keep up quite a chronic course, but Sulphur will clear it up. If, however, it is a double pneumonia, or the hepatization involves a considerable portion of the lung, and the remedy given has not been sufficient, and the case is advancing towards a fatal issue, it may be that all at once at one, two or three o'clock in the morning, he begins to sink, his nose becomes pinched, his lips are drawn, he takes on a hippocratic countenance, is covered with cold sweat, be is too feeble in every part of his body to move; he only moves his head a little in a restless manner.

Unless you are called at once and give him a dose of Arsenicum he will die. You give the Arsenicum, and you have done well, but Arsenicum has no ability to remove the results of inflammation. But though it cannot cure that hepatized lung it acts as a vital stimulant; it warms up the patient and makes him feel he is going to get better; but, mark this, in twenty four hours he will die unless you follow the Arsenicum with the proper remedy.

You must not wait on your remedy too long in these cases. Just as soon as he rallies and the reaction is at its highest pitch, give him the antidote and natural follower of Arsenicum, which is Sulphur, and in twenty-four hours the patient will say,

"I am getting better."

As sure as you exist today, it will do just that thing. There are times when you will see clearly that Phosphorus is the medicine to follow Arsenicum with. If such a patient, rallying under Arsenicum, goes into a fever, if a hot fever comes on with burning thirst and he cannot get enough ice-cold water, you must follow it with Phosphorus, and it will do in that case what Sulphur will do in the other.

You will not see these cases in your own practice because you will not let your cases get into that state; if such cases have power enough to live when prescribed for properly in that state, they have power enough to let you break up the whole nature of the case in the beginning

But go back to that patient who had only a circumscribed hepatization and felt well enough to get up and go around. He has a lingering cough, and now six months or a year after the attack he says.

"Doctor, I have never been right since I had an attack of chest trouble.

The doctor called it pneumonia."

He can tell you about the rusty sputum and the other little things that belong to pneumonia; that is all you need to know. He has had a chronic cough ever since that attack and now he has chilliness.

There is fibrinous infiltration, not a tuberculous state, but the remains of hepatization that nature could not cure. If that is allowed to go on be will go into catarrhal phthisis, asthmatic conditions of chronic bronchitis and troubles of various sorts, and finally he will die from these. Sulphur will very often conform to all of his symptoms; it especially has the ability to clear up the lungs that were not properly cleared up at the time of his illness.

Sulphur cures bronchitis. It cures asthmatic bronchitis when the symptoms agree. Sulphur has a most violent cough that racks the whole frame; it seems that the head will fly off; pain in the head when coughing; the head is jarred by the cough. Then he has expectoration of blood, bleeding from the lungs; in all of these cases threatening phthisis, when there is yet not too much deposit of tubercle, when there is only the beginning of tubercular deposit.

The low, stricken down constitution, the emaciated subjects that have inherited phthisis who have the all-gone hungry feeling in the stomach, heat on the top of the head and uneasiness from the warmth of the bed.

These cases would be better if they had plenty of eruptions come out upon the body; but as a matter of fact the skin has no eruptions; there is no relief; it is all going on in his internals and he is gradually breaking down.

Sulphur will in such instances rouse that patient out of his phthisical state and he will return to health, or, if be is too bad for that be may be kept for years from his troubles. Look out for it in the advanced state of phthisis. You have had sufficient said concerning its administration in such a condition. It increases the suppuration, and brings on little pneumonias wherever there is a tubercle; it tends to suppurate these out. Every cell that is incapable of carrying on its function will be sloughed out by Sulphur.

Back and limbs: The striking thing in Sulphur as to the back is pain in the back on rising from a seat, compelling him to walk bent, and he can only straighten up slowly after moving. The pain is principally in the lumbo-sacral region.

The extremities are covered with eruptions. Eruptions upon the back of the bands and between the fingers, and sometimes upon the palms; vesicular and scaly eruptions which itch; pustules, boils and little abscesses irregular erysipelatous patches here and there upon the extremities a dirty appearance of the skin.

Skin: Itching of the skin from the warmth of the bed.

Enlargement of the joints. Rheumatic affections; great stiffness of the joints; tightness in the hollow of the knees; tightness of the tendons, of rheumatic and gouty character. Cramps in the legs and soles of the feet. Burning of the soles of the feet in bed; he puts them out of bed to cool them of the soles cramp and burn and itch.

At times you will find the soles are cold, and then again burning, and these states alternate with each other. Distress of the body with coldness of the limbs, but after going to bed they burn so much that he must put them out. The corns, which he is a victim of and suffers from almost constantly, burn and sting in the warmth of the bed.

The skin of a Sulphur patient ulcerates and suppurates easily; a splinter under the skin will cause it to ulcerate; wounds heal slowly and fester. Every little prick of a pin festers as in Hepar.

The eruptions of Sulphur are too numerous to mention. They are of all sorts, but there are a few characterizing features in all, such as the burning, stinging and itching and the aggravation from the warmth of the bed.

The skin is rough and unhealthy. Upon the face are many "black-heads," acne, pimples and pustules. Sulphur is full of boils and abscesses in all parts of the body, squamous eruptions, vesicular eruptions, etc.

They are all present in Sulphur and they burn and sting.

 """]

In [ ]:
from torch.nn import functional as F


new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=128)
new_dataset = Dataset(new_encodings, labels=np.ones(len(new_texts), dtype=int))

new_predictions = trainer.predict(new_dataset)

logits = new_predictions.predictions

probs = F.softmax(torch.tensor(logits), dim=-1)

top_k = 97
predicted_classes = torch.topk(probs, top_k, dim=1).indices.tolist()
predicted_labels = [le.inverse_transform(pc) for pc in predicted_classes]

for i, text in enumerate(new_texts):
    print(f"Top {top_k} predicted labels: {predicted_labels[i]}")


Top 97 predicted labels: ['Chocolate' 'Lachesismutus' 'Calcareacarbonica' 'Natriumsulphuricum'
 'Rheumpalmatum' 'Droserarotundifolia' 'Lacvaccinumdefloratum'
 'Cannabisindica' 'Niccolummet+c(oldabbr)' 'Lycopodiumclavatum'
 'Calcareaiodata' 'Sulphur' 'Parisquadrifolia' 'Liliumtigrinum'
 'Mercuriussolubilis' 'Zincummetallicum' 'Plumbummetallicum'
 'Sulphuricumacidum' 'Kaliumarsenicosum' 'Siliceaterra'
 'Rhustoxicodendron' 'Hyoscyamusniger' 'Magnesiummuriaticum' 'Medorrhinum'
 'Hurabrasiliensis' 'Phosphorus' 'Cicutavirosa' 'Pulsatillapratensis'
 'Picricumacidum' 'Najatripudians' 'Cantharisvesicatoria'
 'Barytaarsenicosa' 'Alumina' 'Magnesiumcarbonicum' 'Laccaninum'
 'Barytacarbonica' 'Platinummetallicum' 'Theridioncurassavicum'
 'Heparsulphur' 'Manganumact+c(oldabbr)' 'Graphites' 'Helleborusniger'
 'Coffeacruda' 'Kaliumphosphoricum' 'Tuberculinumbovinumkent'
 'Carcinosinum' 'Strontiumcarbonicum' 'Stramonium' 'Aurummetallicum'
 'Kaliumsulphuricum' 'Violaodorata' 'Nuxvomica' 'Tarentulahispa